In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(1,32,kernel_size=3,padding=1)
    self.conv2 = nn.Conv2d(32,64,kernel_size=3,padding=1)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(64*14*14,128)
    self.fc2 = nn.Linear(128,10)

  def forward(self,X):
    X = nn.functional.relu(self.conv1(X))
    X = nn.functional.relu(self.conv2(X))
    X = nn.functional.max_pool2d(self.dropout1(X),2)
    X = torch.flatten(X,1)
    X = nn.functional.relu(self.fc1(self.dropout2(X)))
    X = self.fc2(X)
    return nn.functional.log_softmax(X,dim=1)

device =  torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
  transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))
])
train_dataset = datasets.MNIST('./data',train=True,download=True,transform=transform)
test_dataset = datasets.MNIST('./data',train=False,transform=transform)

batch_size=64
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()
for batch_idx,(data,target) in enumerate(train_loader):
  data,target = data.to(device),target.to(device)
  optimizer.zero_grad()
  output = model(data)
  loss = nn.functional.nll_loss(output,target)
  loss.backward()
  optimizer.step()

model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
  for data,target in test_loader:
    data,target = data.to(device),target.to(device)
    output = model(data)
    # get the index of the max log probability 
    test_loss += nn.functional.nll_loss(output,target,reduction='sum').item()
    pred = output.argmax(dim=1,keepdim=True)
    correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)


/usr/local/lib/python3.12/dist-packages/torch/nn/functional.py:1531: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [8]:
print(test_loss)

0.05021203028857708
